In [15]:
import sys

sys.path.append("nettack-master")

In [97]:
import json
import pickle
import random
from glob import glob
from itertools import repeat
from pathlib import Path
from uuid import uuid4

import numpy as np
import tensorflow as tf
from joblib import Parallel, delayed
from tqdm import tqdm

In [4]:
tf.test.is_gpu_available()

True

In [5]:
from nettack import GCN
from nettack import nettack as ntk
from nettack import utils

In [6]:
def select_subset_of_keys(dict_, *args):
    return dict((k, dict_[k]) for k in args)

In [7]:
def prepare_surrogate_model_params():
    _A_obs, _X_obs, _z_obs = utils.load_npz("./nettack-master/data/cora.npz")
    _A_obs = _A_obs + _A_obs.T
    _A_obs[_A_obs > 1] = 1
    lcc = utils.largest_connected_components(_A_obs)

    _A_obs = _A_obs[lcc][:, lcc]

    assert np.abs(_A_obs - _A_obs.T).sum() == 0, "Input graph is not symmetric"
    assert (
        _A_obs.max() == 1 and len(np.unique(_A_obs[_A_obs.nonzero()].A1)) == 1
    ), "Graph must be unweighted"
    assert _A_obs.sum(0).A1.min() > 0, "Graph contains singleton nodes"

    _X_obs = _X_obs[lcc].astype("float32")
    _z_obs = _z_obs[lcc]
    _N = _A_obs.shape[0]
    _K = _z_obs.max() + 1
    _Z_obs = np.eye(_K)[_z_obs]
    _An = utils.preprocess_graph(_A_obs)
    sizes = [16, _K]
    degrees = _A_obs.sum(0).A1

    seed = 420
    unlabeled_share = 0.8
    val_share = 0.1
    train_share = 1 - unlabeled_share - val_share
    np.random.seed(seed)

    split_train, split_val, split_unlabeled = utils.train_val_test_split_tabular(
        np.arange(_N),
        train_size=train_share,
        val_size=val_share,
        test_size=unlabeled_share,
        stratify=_z_obs,
    )

    return locals()

In [8]:
surrogate_params = prepare_surrogate_model_params()

Selecting 1 largest connected components


In [9]:
np.random.seed()  # reseed again since prepare_surrogate_model_params set the seed to something

In [10]:
models_weights_path = Path("./weights/")

In [115]:
pickle.dump(surrogate_params, open('surrogate_params.pickle', 'wb'))

In [11]:
def train_surrogate_and_save_weights(surrogate_params, gpu_id):
    with tf.device(f"/gpu:{gpu_id}"):
        surrogate_model = GCN.GCN(
            surrogate_params["sizes"],
            surrogate_params["_An"],
            surrogate_params["_X_obs"],
            with_relu=False,
            name="surrogate",
            gpu_id=0,
        )
        surrogate_model.train(
            surrogate_params["split_train"],
            surrogate_params["split_val"],
            surrogate_params["_Z_obs"],
        )
        W1 = surrogate_model.W1.eval(session=surrogate_model.session)
        W2 = surrogate_model.W2.eval(session=surrogate_model.session)

        surrogate_model.session.close()  # let tf free memory

        path = models_weights_path / str(uuid4())
        np.savez(str(path), W1=W1, W2=W2)
        return path

In [13]:
for _ in tqdm(range(200)):
    train_surrogate_and_save_weights(surrogate_params, 0)

  0%|          | 0/100 [00:00<?, ?it/s]

converged after 52 iterations


  2%|▏         | 2/100 [00:02<02:10,  1.33s/it]

converged after 49 iterations


  3%|▎         | 3/100 [00:03<01:55,  1.19s/it]

...

converged after 56 iterations


 99%|█████████▉| 99/100 [01:46<00:01,  1.13s/it]

converged after 51 iterations


100%|██████████| 100/100 [01:47<00:00,  1.07s/it]

converged after 65 iterations


# NETTACK time Major League Gamers!

In [90]:
dataset_path = Path("./dataset/")

In [108]:
def get_nettack_params(surrogate_params):
    u = random.choice(
        surrogate_params["split_unlabeled"]
    )  # randomly pick an attacked node
    return {
        "u": u,
        "direct_attack": False,
        "n_influencers": int(surrogate_params["degrees"][u]),
        # limit the number of perturbations, since the computation takes too much time
        # (max node degree is around 160)
        "n_perturbations": min(50, int(surrogate_params["degrees"][u])),
        "perturb_features": True,
        "perturb_structure": True,
    }

In [109]:
def perform_nettack_on_a_surrogate_graph(surrogate_params, weights_path):
    nettack_params = get_nettack_params(surrogate_params)
    weigths = np.load(weights_path)

    nettack = ntk.Nettack(
        surrogate_params["_A_obs"],
        surrogate_params["_X_obs"],
        surrogate_params["_z_obs"],
        weigths["W1"],
        weigths["W2"],
        nettack_params["u"],
        verbose=False,
    )

    nettack.reset()

    nettack.attack_surrogate(
        nettack_params["n_perturbations"],
        perturb_structure=nettack_params["perturb_structure"],
        perturb_features=nettack_params["perturb_features"],
        direct=nettack_params["direct_attack"],
        n_influencers=nettack_params["n_influencers"],
    )

    out = {
        "params": nettack_params,
        "structure_perturbations": nettack.structure_perturbations,
        "feature_perturbations": nettack.feature_perturbations,
    }
    with open(dataset_path / f"{uuid4()}.pickle", "wb") as f:
        pickle.dump(out, f)

In [110]:
# perform_nettack_on_a_surrogate_graph(
#     surrogate_params, "./weights/00c1590c-cd03-4351-97cb-503c5a841224.npz"
# )

In [111]:
def dataset_generator():
    for weights_path in glob("./weights/*.npz"):
        for _ in range(100):
            yield weights_path

In [ ]:
Parallel(n_jobs=32, verbose=10)(
    delayed(perform_nettack_on_a_surrogate_graph)(surrogate_params, weights_path)
    for weights_path in dataset_generator()
)

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:   22.1s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:   26.0s
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:   31.6s
[Parallel(n_jobs=32)]: Done  49 tasks      | elapsed:   37.6s
[Parallel(n_jobs=32)]: Done  64 tasks      | elapsed:   42.7s
[Parallel(n_jobs=32)]: Done  81 tasks      | elapsed:   52.7s
[Parallel(n_jobs=32)]: Done  98 tasks      | elapsed:   58.8s
[Parallel(n_jobs=32)]: Done 117 tasks      | elapsed:  1.2min
[Parallel(n_jobs=32)]: Done 136 tasks      | elapsed:  1.3min
[Parallel(n_jobs=32)]: Done 157 tasks      | elapsed:  1.6min
[Parallel(n_jobs=32)]: Done 178 tasks      | elapsed:  1.8min
[Parallel(n_jobs=32)]: Done 201 tasks      | elapsed:  2.1min
[Parallel(n_jobs=32)]: Done 224 tasks      | elapsed:  2.4min
[Parallel(n_jobs=32)]: Done 249 tasks      | elapsed:  2.7min
[Parallel(n_jobs=32)]: Done 274 tasks      | elapsed:  